In [5]:
import os
import numpy as np
import pandas as pd
from pandas import read_csv
import geopandas as gpd
import fiona
import rasterio as rio
from tqdm import tqdm
import calendar
import datetime


In [6]:
## stack ALL VIs in a folder. if return_csv is True - output points too 

## TO-DO:
## CRS check that reprojects polys to raster crs

def stack_extract(input_dir, out_name, inputShape, return_csv=True):
    
    rasList = []
    bandList = []
    
    for img in sorted(os.listdir(input_dir)):
        if img.endswith('.tif'):
            rasList.append(os.path.join(input_dir,img))
    print('number of images: ', len(rasList))
    band_paths=rasList
    # Read in metadata
    first_band = rio.open(band_paths[0], 'r')
    meta = first_band.meta.copy()
    # Replace metadata with new count and create a new file
    counts = 0
    for ifile in band_paths:
        with rio.open(ifile, 'r') as ff:
            counts += ff.meta['count']
    meta.update(count=counts)
    out_path = out_name + ".tif"
    with rio.open(out_path, 'w', **meta) as ff:
        for ii, ifile in tqdm(enumerate(band_paths)):
            bands = rio.open(ifile, 'r').read()
            if bands.ndim != 3:
                bands = bands[np.newaxis, ...]
            for band in bands:
                ff.descriptions = tuple([i[-11:-4] for i in band_paths])
                ff.write(band, ii+1)
            bandList.append(ff.descriptions)
            
    plots = []            
    multi_values_points = pd.Series()
    with fiona.open(inputShape) as shp:
        for feature in shp:
            siteID = feature['properties']['Target_FID']
            ProdID = feature['properties']['oldIDPROD']
            Parcela = feature['properties']['Parcela']
            coords = feature['geometry']['coordinates']
            # Read pixel value at the given coordinates using Rasterio
            # NB: `sample()` returns an iterable of ndarrays.
            with rio.open(out_path) as stack_src:            
                value = [v for v in stack_src.sample([coords])] 
            # Update the pandas series accordingly
            multi_values_points.loc[siteID] = value
            plots.append(int(siteID))
            plots.append(ProdID)
            plots.append(Parcela)
            
    plots_arr = np.array(plots)
    plots_arr_reshape = plots_arr.reshape(int(plots_arr.shape[0]/3), -1)
    plots_df = pd.DataFrame(plots_arr_reshape, columns = ['FID','FieldID','Parcela'])
    df1 = pd.DataFrame(multi_values_points.values, index=multi_values_points.index)
    df1.columns = ['Val']
    df2 = pd.DataFrame(df1['Val'].explode())
    extracted_vals = pd.DataFrame(df2["Val"].to_list())
    extracted_vals.columns = bandList[0]
    extracted_vals['FID'] = np.arange(len(extracted_vals)).astype(str)
    extracted_plots = pd.merge(plots_df, extracted_vals, how = 'outer', on='FID')
    extracted_plots =  extracted_plots.set_index("FID")
    single_grid_vals = extracted_plots[~(extracted_plots.T == 0).any()]
    #print(single_grid_vals) # print if nothing is outputting, export as wide format
    ## REFORMAT
    df2 = single_grid_vals.set_index(["FieldID", "Parcela"])
    df2_T = df2.T
    df3 = df2_T.reset_index(level=0)
    df3["Julian"] = df3["index"].str.slice(start=0, stop=9).astype(str)
    df3["Date"] = pd.to_datetime(df3['Julian'], format='%Y%j')
    df4 = df3.set_index('Date')
    df5 = df4.drop(['index', 'Julian'], axis=1)
    df5 =  df5.reset_index(level=0) 
    df5_melt = pd.melt(df5, id_vars='Date')
    df5_melt.rename(columns = {'Date':'Date', 'ProdID': 'Code', 'Parcela': 'Parcela', 'value': 'Value'}, inplace=True)
    #print(df5_melt.to_string())

    if return_csv is True:
        out_csv_path = out_name + ".csv"
        print('exporting csv: ' + out_csv_path)
        return df5_melt.to_csv(out_csv_path)
    else:
        return('no csv for ' + out_csv_path)


In [6]:
index_list = ['wi'] # ['evi2', 'gcvi', 'kndvi', 'nbr', 'wi', 'ndti',]

inputShape_ALBERS = '/home/sandbox-cel/wsa_lc/WSA_AOI/CRS_Plot_Locations/NI_ES_points_AEA.shp'
inputShape_UTM = '/home/sandbox-cel/wsa_lc/WSA_AOI/CRS_Plot_Locations/NI_ES_points_UTM16.shp'

ALB_NI_grids_for_NDTI = ['002225','002387','002550']
UTM_NI_grids_for_NDTI = ['002306', '002307', '002386', '002388', '002548', '002549']

grid_list_ALBERS = ['001892','001972','002047','002056','002127','002225','002387','002550']
grid_list_UTM = ['002306','002307','002386'] # 002047,002127 ## '002306','002307','002386','002388','002548','002549'


In [7]:
index_list = ['ndti'] # ['evi2', 'gcvi', 'kndvi', 'nbr', 'wi', 'ndti',]

ALB_NI_grids_for_NDTI = ['002225','002387','002550']
UTM_NI_grids_for_NDTI = ['002306', '002307', '002386', '002388', '002548', '002549']

inputShape_ALBERS = '/home/sandbox-cel/wsa_lc/WSA_AOI/CRS_Plot_Locations/NI_ES_points_AEA.shp'
inputShape_UTM = '/home/sandbox-cel/wsa_lc/WSA_AOI/CRS_Plot_Locations/NI_ES_points_UTM16.shp'


for grid in ALB_NI_grids_for_NDTI: ## change _ALBERS or UTM
    for index in index_list:
        input_dir = "/home/downspout-cel/wsa_lc/raster/grids/" + grid + "/brdf_ts/ms/" + index + "/" ### crs_lc/wsa_lc for UTM/Albers
        out_name = "/home/downspout-cel/wsa_lc/TS/final/" + grid + "_" + index + "_TimeSeries"
        stack_extract(input_dir, out_name, inputShape=inputShape_ALBERS, return_csv=True) ## change _ALBERS or UTM  
        
for grid in UTM_NI_grids_for_NDTI: ## change _ALBERS or UTM
    for index in index_list:
        input_dir = "/home/downspout-cel/crs_lc/raster/grids/" + grid + "/brdf_ts/ms/" + index + "/" ### crs_lc/wsa_lc for UTM/Albers
        out_name = "/home/downspout-cel/wsa_lc/TS/final/" + grid + "_" + index + "_TimeSeries"
        stack_extract(input_dir, out_name, inputShape=inputShape_UTM, return_csv=True) ## change _ALBERS or UTM
        

number of images:  245


245it [02:44,  1.49it/s]
/home/lsharwood/.nasaenv/lib64/python3.6/site-packages/ipykernel_launcher.py:37: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/home/lsharwood/.nasaenv/lib64/python3.6/site-packages/pandas/core/generic.py:3889: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


exporting csv: /home/downspout-cel/wsa_lc/TS/final/002225_ndti_TimeSeries.csv
number of images:  246


246it [02:52,  1.42it/s]


exporting csv: /home/downspout-cel/wsa_lc/TS/final/002387_ndti_TimeSeries.csv
number of images:  246


...

exporting csv: /home/downspout-cel/wsa_lc/TS/final/002388_ndti_TimeSeries.csv
number of images:  246


246it [03:14,  1.26it/s]


exporting csv: /home/downspout-cel/wsa_lc/TS/final/002548_ndti_TimeSeries.csv
number of images:  246


246it [03:10,  1.29it/s]


exporting csv: /home/downspout-cel/wsa_lc/TS/final/002549_ndti_TimeSeries.csv
